In [2]:
import math

import numpy as np
from scipy import signal
import utils
import mediapipe as mp
import cv2
from scipy.spatial import ConvexHull

In [5]:
def generateMaskFromPoints(ROI, face, w, h):
    points = []
    for current_point in ROI:
        x = int(face.landmark[current_point].x * w)
        y = int(face.landmark[current_point].y * h)
        points.append([x, y])

    mask = np.zeros((h, w), dtype=np.uint8)
    points = np.array(points, dtype=np.int32)
    cv2.fillConvexPoly(mask, points, 255)

    return mask

In [27]:
mp_face_mesh = mp.solutions.face_mesh

video_path = "Dataset/Patient_1/Q1_1/vid_crop.avi"
#video_path = "Dataset/Patient_2/Q1_1/vid_crop.avi"

cap = cv2.VideoCapture(video_path)

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
)

class ROIS: 

        OJO_IZQ = [26, 22, 23, 24, 110, 25, 130, 247, 30, 29, 27, 28, 56, 190, 243, 112]

        OJO_DER = [253, 254, 339, 255, 359, 467, 260, 259, 257, 258, 286, 414, 463, 341, 256, 252]

        BOCA = [57, 186, 92, 165, 167, 164, 393, 391, 322, 410, 287, 273, 335, 406, 313, 18, 83, 182, 106, 43]

        FULL_FACE = [175, 171, 140, 170, 169, 135, 138, 215, 177, 137, 227, 34, 139, 71, 54, 103, 67, 109, 10, 338, 297, 332, 284,
         301, 368, 264, 447, 366, 401, 435, 367, 364, 394, 395, 369, 396]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Generate the skin mask
    #########################3
    h, w, _ = frame.shape

    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    if not results.multi_face_landmarks:
        print("Error A")
        break

    face = results.multi_face_landmarks[0]
    
    # final_mask = np.zeros((h, w), dtype=np.uint8)
    # for ROI in ROIS:
    #     current_mask = generateMaskFromPoints(ROI, face, w, h)
    #     final_mask = cv2.bitwise_or(final_mask, current_mask)

    mask_ojos = cv2.bitwise_or(
        generateMaskFromPoints(ROIS.OJO_IZQ, face, w, h),
        generateMaskFromPoints(ROIS.OJO_DER, face, w, h),
    )

    mask_excluded = cv2.bitwise_or(mask_ojos, generateMaskFromPoints(ROIS.BOCA, face, w, h))

    mask_full = generateMaskFromPoints(ROIS.FULL_FACE, face, w, h)

    mask_excluded_inv = cv2.bitwise_not(mask_excluded)
    final_mask = cv2.bitwise_and(mask_full, mask_excluded_inv)

    # Make a copy and black out all pixels outside the mask
    masked_frame = np.zeros_like(frame_rgb)
    masked_frame[final_mask > 0] = frame_rgb[final_mask > 0]

    # Show
    cv2.imshow("Skin Mask Only", cv2.cvtColor(masked_frame, cv2.COLOR_RGB2BGR))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 